In [ ]:
import sys, math
import numpy as np
sys.setrecursionlimit(100000)

In [ ]:
lines = """\
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output""".strip().split('\n')
lines

In [ ]:
lines = """\
%bc -> ph
%hr -> zq
%sn -> sj
%df -> dn, hf
%lp -> dm
%lf -> sn, vk
&fv -> sq
%gd -> vm, gr
%jt -> vm, lz
%xf -> vk
%nf -> lp, kb
%dl -> nn
&sq -> rx
%vb -> vm, lg
%zq -> kb, fl
%fk -> fz
%gj -> pq
%jx -> jh
%pq -> kb, nf
&dn -> kg, vt, tg, fs, pn, jx
%ln -> dn, fs
%fz -> vm, jt
%fs -> df
%dm -> kb, hr
%ds -> kb
&kk -> sq
%tg -> tv
&vt -> sq
%fl -> zl, kb
&vk -> bc, sj, jd, lf, xr, sn
%jd -> pp
%tv -> dn, pn
%sb -> gf, vk
&kb -> kk, gj, gt, hr, lp
%pp -> vk, bc
%pn -> pf
%zc -> vm
&vm -> dl, fk, nn, fv, gd, lg
%rn -> dn
%gr -> vb, vm
%sj -> kq
%zl -> kb, ds
%lz -> vm, zc
%jh -> dn, ln
%pf -> dn, jx
%kq -> sb, vk
%ph -> md, vk
%nc -> gj, kb
%kg -> tg, dn
%hf -> dn, rn
%nn -> fk
%gf -> jd, vk
%lg -> pv
broadcaster -> gd, kg, gt, lf
%gt -> nc, kb
%pv -> dl, vm
&xr -> sq
%md -> vk, xf""".strip().split('\n')
lines

In [ ]:
graph = {}
for line in lines:
    parts = line.split(' -> ')
    graph[parts[0]] = parts[1].split(', ')
for key, targets in graph.items():
    graph[key] = [ f"%{target}"  if f"%{target}" in graph else f"&{target}" for target in targets ]
graph

In [ ]:
states = {}
for key in graph:
    if key.startswith("%"): # flipflow. start as off and flips when receiving low. when flips on sends a high, turn off sends low
        states[key] = False
for conkey in graph:
    if conkey.startswith("&"):
        states[conkey] = dict()
        for key, value in graph.items():
            if conkey in value:
                states[conkey][key] = False
states

In [ ]:
states = {}
for key in graph:
    if key.startswith("%"): # flipflow. start as off and flips when receiving low. when flips on sends a high, turn off sends low
        states[key] = False
for conkey in graph:
    if conkey.startswith("&"):
        states[conkey] = dict()
        for key, value in graph.items():
            if conkey in value:
                states[conkey][key] = False

v = 0 # verbosity
nlow = 0
nhigh = 0
S = {True:"high", False:"low"}
for _ in range(1000):
    queue = [("broadcaster", False, None)] # curr, signal, module where signal is coming from
    while queue:
        (module, signal, mfrom) = queue.pop(0)

        if signal:
            nhigh += 1
        else:
            nlow += 1

        try:
            targets = graph[module]
        except KeyError as e:
            #print("K", end="")
            continue
        state = states.get(module, None)

        if v: print(f"at M:{module} Sig:{signal} @{state}")

        if module.startswith("%"):
            if not signal: # low signal -> switch
                if state: # if on, go off and send low
                    states[module] = False
                    new_signal = False
                else: # if off, switch on, send high
                    states[module] = True
                    new_signal = True

                for target in targets:
                    if v: print(f"{module} sends {S[new_signal]} to {target}") 
                    queue.append((target, new_signal, module))
        elif module.startswith("&"): #if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.
            states[module][mfrom] = signal # When a pulse is received, the conjunction module first updates its memory for that input.
            if v: print(state, state.values())
            if all(state.values()):
                new_signal = False
            else:
                new_signal = True
            for target in targets:
                if v: print(f"{module} sends {S[new_signal]} to {target}") 
                queue.append((target, new_signal, module))
        else: # for broadcaster
            for target in targets:
                if v: print(f"{module} sends {S[signal]} to {target}") 
                queue.append((target, signal, module))

print(nlow, nhigh)
answer = nlow * nhigh
print(f"Part 1: {answer}")

## question 2

In [ ]:
states = {}
for key in graph:
    if key.startswith("%"): # flipflow. start as off and flips when receiving low. when flips on sends a high, turn off sends low
        states[key] = False
for conkey in graph:
    if conkey.startswith("&"):
        states[conkey] = dict()
        for key, value in graph.items():
            if conkey in value:
                states[conkey][key] = False

# A little bit of hardcoded trickery for my input... I specifically needed xr, fv, kk, and vt to all be low.
lowest_parents = {
    "&xr": None,
    "&fv": None,
    "&kk": None,
    "&vt": None,
}

v = 0 # verbosity
nlow = 0
nhigh = 0
S = {True:"high", False:"low"}
nit = 0
maxit = 100000
while True:
    if all(val is not None for val in lowest_parents.values()):
        answer = math.lcm(*list(lowest_parents.values()))
        break

    nit += 1
    if nit >= maxit:
        print("reached max iterations")
        break
    
    queue = [("broadcaster", False, None)] # curr, signal, module where signal is coming from
    while queue:
        (module, signal, mfrom) = queue.pop(0)

        if signal:
            nhigh += 1
        else:
            nlow += 1

        if module in lowest_parents and not signal:
            print("---yippy!")
            lowest_parents[module] = nit

        # Ideally this happens but it won't.
        if module == "rx" and not signal:
            break

        try:
            targets = graph[module]
        except KeyError as e:
            if v: print("K", end="")
            continue
        state = states.get(module, None)

        if v: print(f"at M:{module} Sig:{signal} @{state}")

        if module.startswith("%"):
            if not signal: # low signal -> switch
                if state: # if on, go off and send low
                    states[module] = False
                    new_signal = False
                else: # if off, switch on, send high
                    states[module] = True
                    new_signal = True

                for target in targets:
                    if v: print(f"{module} sends {S[new_signal]} to {target}") 
                    queue.append((target, new_signal, module))
        elif module.startswith("&"): #if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.
            states[module][mfrom] = signal # When a pulse is received, the conjunction module first updates its memory for that input.
            if v: print(state, state.values())
            if all(state.values()):
                new_signal = False
            else:
                new_signal = True
            for target in targets:
                if v: print(f"{module} sends {S[new_signal]} to {target}") 
                queue.append((target, new_signal, module))
        else: # for broadcaster
            for target in targets:
                if v: print(f"{module} sends {S[signal]} to {target}") 
                queue.append((target, signal, module))

lowest_parents, answer